# Qiskit Assignment 1
## Practice with Pauli Gates and other Qiskit tools
Welcome to your second Qiskit assignment!

### Learning Objectives
1. Build Pauli gates from Qiskit's U gate
2. Use the U gate to reverse a series of operations
3. Visualize rotations using the Bloch Sphere

In [ ]:
# Import Qiskit and other needed packages
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector,visualize_transition
from qiskit.quantum_info import Statevector
import numpy as np
import pprint
import random

#### Task 1 - Constructing Pauli Z gate from u Gate
We can use [Qiskit's U Gate](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.u.html#qiskit.circuit.QuantumCircuit.u) to construct arbitrary quantum operations. Fill in the function below to return `qc_pauli_z`, a QuantumCircuit satisfying the following conditions:
- it has 1 qubit, [initialized](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.initialize.html#qiskit.circuit.QuantumCircuit.initialize) to the parameter `initial_state`
- it has 1 classical bit
- it has a U gate with parameters which perform the same rotation as a Pauli Z gate
- it does not use the built in Z gate
- it performs a measurement following the rotation

In [ ]:
def qc_pauli_z(initial_state=[1,0]):
    # BEGIN SOLUTION
    theta = 0
    phi = np.pi
    lamda = 0
    
    qc_pauli_z = QuantumCircuit(1,1)
    qc_pauli_z.initialize(initial_state, 0)
    qc_pauli_z.u(theta, phi, lamda, 0)
    qc_pauli_z.measure(0,0)
    # END SOLUTION
    return qc_pauli_z

You can use the following code to help visualize your circuit.

In [ ]:
qc_pauli_z().draw(output='mpl')

In [ ]:
qc_pauli_z().num_clbits == 1

In [ ]:
qc_pauli_z().num_qubits == 1

In [ ]:
def testNoQiskitZGate():
    ops = qc_pauli_z().count_ops()
    try:
        find_z = ops['z']
    except KeyError: # this is the passing case, we don't want Z to be found!
        return True
    else:
        return False
    
testNoQiskitZGate()

In [ ]:
def testInitialState():
    try:
        num_initializations = qc_pauli_z().count_ops()['initialize']
    except KeyError:
        return False
    else:
        return num_initializations == 1
    
testInitialState()

In [ ]:
def testMeasurementPerformed():
    try:
        num_measurements = qc_pauli_z().count_ops()['measure']
    except KeyError:
        return False
    else:
        return num_measurements == 1

testMeasurementPerformed()

In [ ]:
def testRandomInitialState():
    # let's choose something interesting as our initial state!
    c = random.uniform(0, 2*np.pi)
    initial_state = [(np.cos(c)), (np.sin(c))]
    pauli_z_matrix = np.array(
                                [[1, 0],
                                 [0,-1]]
                             )
    
    qc = qc_pauli_z(initial_state)
    qc.remove_final_measurements()
    result_sv = Statevector.from_instruction(qc)
    
    return result_sv.equiv(Statevector(pauli_z_matrix.dot(initial_state)))

testRandomInitialState()

In [ ]:
# HIDDEN
def testSolutionPauliZ():
    # choose something interesting!
    c = random.uniform(0, 2*np.pi)
    initial_state = [(np.cos(c)), (np.sin(c))]

    theta = 0
    phi = np.pi
    lamda = 0
    
    solution_qc = QuantumCircuit(1,1)
    solution_qc.initialize(initial_state, 0)
    solution_qc.z(0)
    
    student_qc = qc_pauli_z(initial_state)
    student_qc.remove_final_measurements()
    return Statevector.from_instruction(student_qc).equiv(Statevector.from_instruction(solution_qc))

testSolutionPauliZ()

#### Task 2 (2A, 2B, 2C) - Unitary Inverse Puzzles

We'll study the idea of [uncomputation](https://qiskit.org/textbook/ch-algorithms/grover.html#5.2-Uncomputing,-and-Completing-the-Oracle) during our discussion of quantum algorithms. In general, we may find it helpful to return a qubit to its initial state. 

This process is typically straightforward due to the properties of unitary gates. However, your task is to do so using only a single U gate. Complete the partial circuits below such that the measurements will yield a state equivalent to `initial_state` *up to a global phase*.

#### Task 2A

In [ ]:
def reverse_a(initial_state=[1,0]):
    qc_reverse_a = QuantumCircuit(1,1)
    qc_reverse_a.initialize(initial_state)
    qc_reverse_a.x(0)
    qc_reverse_a.h(0)
    qc_reverse_a.y(0)
    qc_reverse_a.x(0)
    qc_reverse_a.z(0)
    qc_reverse_a.barrier()
    
   # BEGIN SOLUTION
    # Can use commutation relation XY = iZ to reduce
    # Find parameters for U(?,?,?) = XH
    theta = 3*np.pi/2
    phi = np.pi
    lamda = np.pi
    qc_reverse_a.u(theta, phi, lamda, 0)
   # END SOLUTION

    qc_reverse_a.barrier()
    qc_reverse_a.measure(0,0)
    return qc_reverse_a

In [ ]:
reverse_a().draw(output='mpl')

In [ ]:
def testAcceptableGateUsage2A():
    ops = reverse_a().count_ops()
    x = ops['x'] == 2
    y = ops['y'] == 1
    z = ops['z'] == 1
    h = ops['h'] == 1
    try:
        u = ops['u'] == 1
    except KeyError:
        u = False
        
    return all([x,y,z,h,u])
    
testAcceptableGateUsage2A()

In [ ]:
def testKetZero2A():
    qc = reverse_a()
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([1,0]))
    
testKetZero2A()

In [ ]:
def testKetOne2A():
    initial_state = [0,1]
    qc = reverse_a(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([0,1]))
    
testKetOne2A()

In [ ]:
def testRandomSimple2A():
    c = random.uniform(0, 2*np.pi)
    initial_state = [(np.cos(c)), (np.sin(c))]
    qc = reverse_a(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector(initial_state))
    
testRandomSimple2A()

In [ ]:
# HIDDEN
def testRandomHidden2A():
    results = []
    for i in range(3):
        c = random.uniform(0, 2*np.pi)
        initial_state = [(np.cos(c)), (np.sin(c))]
        qc = reverse_a(initial_state)
        qc.remove_final_measurements()
        sv = Statevector.from_instruction(qc)
        results.append(sv.equiv(Statevector(initial_state)))
    return all(results)
    
testRandomHidden2A()

#### Task 2B

In [ ]:
def reverse_b(initial_state=[1,0]):
    qc_reverse_b = QuantumCircuit(1,1)
    qc_reverse_b.initialize(initial_state)
    for i in range(5):
        qc_reverse_b.x(0)
        qc_reverse_b.y(0)
        qc_reverse_b.z(0)
        qc_reverse_b.h(0)
    qc_reverse_b.barrier()
    
    
    # BEGIN SOLUTION
    # Use commutation relation YX = -iZ
    # so that (HZYX)^5=(-iHZZ)^5=(-iH)^5=-iH
    # Hence to measure state within a global we use
    # parameters for the U gate such that U == H
    theta = np.pi/2
    phi = 0
    lamda = np.pi
    qc_reverse_b.u(theta, phi, lamda, 0)
    # END SOLUTION

    qc_reverse_b.barrier()
    qc_reverse_b.measure(0,0)
    return qc_reverse_b

In [ ]:
reverse_b().draw(output='mpl')

In [ ]:
def testAcceptableGateUsage2B():
    ops = reverse_b().count_ops()
    x = ops['x'] == 5
    y = ops['y'] == 5
    z = ops['z'] == 5
    h = ops['h'] == 5
    try:
        u = ops['u'] == 1
    except KeyError:
        u = False
    return all([x,y,z,h,u])
    
testAcceptableGateUsage2B()

In [ ]:
def testKetZero2B():
    qc = reverse_b()
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([1,0]))
    
testKetZero2B()

In [ ]:
def testKetOne2B():
    initial_state = [0,1]
    qc = reverse_b(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([0,1]))
    
testKetOne2B()

In [ ]:
def testRandomSimple2B():
    c = random.uniform(0, 2*np.pi)
    initial_state = [(np.cos(c)), (np.sin(c))]
    qc = reverse_b(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector(initial_state))
    
testRandomSimple2B()

In [ ]:
# HIDDEN
def testRandomHidden2B():
    results = []
    for i in range(3):
        c = random.uniform(0, 2*np.pi)
        initial_state = [(np.cos(c)), (np.sin(c))]
        qc = reverse_b(initial_state)
        qc.remove_final_measurements()
        sv = Statevector.from_instruction(qc)
        results.append(sv.equiv(Statevector(initial_state)))
    return all(results)
    
testRandomHidden2B()

#### Task 2C

**Hint**: The P gate generalizes rotation about the Z-axis to an arbitrary angle $\phi$, where
$P(\phi)=\begin{pmatrix} 1 & 0 \\ 0 & e^{i\phi} \end{pmatrix}$

In [ ]:
def reverse_c(initial_state=[1,0]):
    qc_reverse_c = QuantumCircuit(1,1)
    qc_reverse_c.initialize(initial_state)
    qc_reverse_c.x(0)
    for i in range(1,6):
        qc_reverse_c.p((-1)**(i)*np.pi/(2**i), 0)
    qc_reverse_c.z(0)
    qc_reverse_c.barrier()
    
    # BEGIN SOLUTION
    # Trick: combine P gates and Z into a single P gate
    # Note that Z == P(pi)
    # Find parameters for U(?,?,?) = (P(-11pi/32)*X)
    theta = np.pi
    phi = np.pi
    lamda = -21*np.pi/32
    qc_reverse_c.u(theta, phi, lamda, 0)
    # END SOLUTION

    qc_reverse_c.barrier()
    qc_reverse_c.measure(0,0)
    return qc_reverse_c

In [ ]:
reverse_c().draw(output='mpl')

In [ ]:
def testAcceptableGateUsage2C():
    ops = reverse_c().count_ops()
    x = ops['x'] == 1
    y = not 'y' in ops
    z = ops['z'] == 1
    h = not 'h' in ops
    p = ops['p'] == 5
    try:
        u = ops['u'] == 1
    except KeyError:
        u = False
    return all([x,y,z,h,p,u])
    
testAcceptableGateUsage2C()

In [ ]:
def testKetZero2C():
    qc = reverse_c()
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([1,0]))
    
testKetZero2C()

In [ ]:
def testKetOne2C():
    initial_state = [0,1]
    qc = reverse_c(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector([0,1]))
    
testKetOne2C()

In [ ]:
def testRandomSimple2C():
    c = random.uniform(0, 2*np.pi)
    initial_state = [(np.cos(c)), (np.sin(c))]
    qc = reverse_c(initial_state)
    qc.remove_final_measurements()
    sv = Statevector.from_instruction(qc)
    return sv.equiv(Statevector(initial_state))
    
testRandomSimple2C()

In [ ]:
# HIDDEN
def testRandomHidden2C():
    results = []
    for i in range(3):
        c = random.uniform(0, 2*np.pi)
        initial_state = [(np.cos(c)), (np.sin(c))]
        qc = reverse_c(initial_state)
        qc.remove_final_measurements()
        sv = Statevector.from_instruction(qc)
        results.append(sv.equiv(Statevector(initial_state)))
    return all(results)
    
testRandomHidden2C()

#### Task 3 (3A, 3B, 3C) - Using Rotation to Obtain Probabilities

#### Task 3A 
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$ 
Fill in the function below to return `qc_rot_a`, a single-qubit QuantumCircuit satisfying the following conditions:
- it performs a measurement to a single classical bit
- Pr(seeing $\ket{0}$ on measurement) = `0.75`
- your circuit only uses gates from the following list: X, Y, Z, P, H, U

Plot your results using a histogram to verify your solution over `1024` trials.

In [ ]:
def qc_rot_a():
    # BEGIN SOLUTION
    # Find wave amplitudes for ket zero and ket one
    # Use amplitudes to determine the proportion of rotation needed 
    qc_rot_a = QuantumCircuit(1,1)
    theta = np.pi/3
    phi = 0
    lamda = 0
    qc_rot_a.u(theta, phi, lamda, 0)
    qc_rot_a.measure(0,0)
    # END SOLUTION
    return qc_rot_a

In [ ]:
qc_rot_a().draw(output='mpl')

In [ ]:
# Plot your results in this cell!

# BEGIN SOLUTION
qc = qc_rot_a()
qasm_sim = BasicAer.get_backend("qasm_simulator")
job = execute(qc, qasm_sim)
counts = job.result().get_counts()
plot_histogram(counts)
# END SOLUTION

#### Task 3B - Rotation Operator Gates
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$ 
Again, fill in the function below to return `qc_rot_b`, a single-qubit QuantumCircuit satisfying the following conditions:
- it performs a measurement to a single classical bit
- Pr(seeing $\ket{0}$ on measurement) = `0.75`
- your circuit only uses gates from the following list: [RX, RY, RZ](https://en.wikipedia.org/wiki/Quantum_logic_gate#Rotation_operator_gates)

Plot your results using a histogram to verify your solution over `1024` trials.

In [ ]:
def qc_rot_b():
    # BEGIN SOLUTION
    # Using the provided link, it can be seen that RX(pi) = -iX
    # From this info and part A, we need to use RX(pi/3) to get the desired state
    qc_rot_b = QuantumCircuit(1,1)
    qc_rot_b.rx(np.pi/3, 0)
    qc_rot_b.measure(0,0)
    # END SOLUTION
    return qc_rot_b

In [ ]:
qc_rot_b().draw(output='mpl')

In [ ]:
# Plot your results in this cell!

# BEGIN SOLUTION
qasm_sim = BasicAer.get_backend("qasm_simulator")
job = execute(qc_rot_b(), qasm_sim)
counts = job.result().get_counts()
plot_histogram(counts)
# END SOLUTION

#### Task 3C
Suppose we apply a Z gate to your circuit from task 3B just before measuring. How will the probability of measuring $\ket{0}$ change from that of the original circuit? Is this rotation factorable from the state as a global phase?

### Extension Ideas
1. Research and explain the difference between the RZ and P gates
2. Prove P=NP